In [140]:
import pandas as pd
import re

In [114]:
thinktank_df = pd.read_csv("./data/thinktank_citations_tdm.csv", index_col=0)

# Data Cleaning

In [164]:
'asf.  '.strip()

'asf.'

In [165]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    no_tags = re.sub(clean, '', text)
    return no_tags.strip()

In [168]:
thinktank_df['sentence_clean'] = thinktank_df['sentence'].apply(remove_html_tags)

In [117]:
thinktank_df['year']=pd.to_datetime(thinktank_df['date']).apply(lambda x: x.year)

# Topic Tagging w/ Keywords

## International Country Mentions

In [118]:
import pycountry

In [119]:
nlp = spacy.load("en_core_web_sm")

In [120]:
def get_international(txt):
    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            try:
                match = pycountry.countries.search_fuzzy(ent.text)[0].name
                if match == 'United States' or match == 'Virgin Islands, U.S.':
                    continue
                return match
            except:
                continue

In [121]:
thinktank_df['international_mention'] = thinktank_df['sentence_clean'].apply(get_international)

In [122]:
thinktank_df.to_csv("./data/thinktank_citations_tagged.csv")

## Tagging Other Topics

In [167]:
thinktank_df = pd.read_csv("./data/thinktank_citations_tagged.csv", index_col=0)

In [180]:
econ_keywords = ['monetary', 'inflation', 'Fed', 'Federal Reserve', 'labor market', 'labor union',
                 'labor', 'tax', 'taxes', 'taxation', 'banking', 'banks', 'finance', 'financial']

education_keywords = ['school', 'schools', 'education', 'college', 'university', 'universities', 'teacher',
                     'teachers', 'kindergarten', 'students', 'student']

health_keywords = ['health insurance', 'Medicare', 'hospital', 'hospitals', 'medicine', 'health care', 
                   'doctor', 'doctors']

politics_keywords = ['Democrat', 'Republican', 'Senate', 'The House', 'caucus', 'Congress', 'bill', 'legislation']

In [181]:
def tag_keywords(txt, keyword_list):
    txt = txt.lower()
    for keyword in keyword_list:
        if keyword.lower() in txt:
            return 1
    return 0

In [182]:
# economy topics
thinktank_df['econ_keyword'] = thinktank_df['sentence_clean'].apply(lambda x: tag_keywords(x, econ_keywords))
# education topics
thinktank_df['edu_keyword'] = thinktank_df['sentence_clean'].apply(lambda x: tag_keywords(x, education_keywords))
# health topics
thinktank_df['health_keyword'] = thinktank_df['sentence_clean'].apply(lambda x: tag_keywords(x, health_keywords))
# political topics
thinktank_df['pol_keyword'] = thinktank_df['sentence_clean'].apply(lambda x: tag_keywords(x, politics_keywords))

In [183]:
thinktank_df[thinktank_df.econ_keyword == 1].groupby(['pub', 'year', 'thinktank']) \
    ['title'].count().reset_index().shape

(854, 4)

In [184]:
thinktank_df[thinktank_df.edu_keyword == 1].groupby(['pub', 'year', 'thinktank']) \
    ['title'].count().reset_index().shape

(744, 4)

In [185]:
thinktank_df[thinktank_df.health_keyword == 1].groupby(['pub', 'year', 'thinktank']) \
    ['title'].count().reset_index().shape

(312, 4)

In [186]:
thinktank_df[thinktank_df.pol_keyword == 1].groupby(['pub', 'year', 'thinktank']) \
    ['title'].count().reset_index().shape

(876, 4)

In [157]:
thinktank_df.to_csv("./data/thinktank_citations_tagged.csv")

In [191]:
thinktank_df[thinktank_df.pol_keyword == 1].sample(1)

,file_path,title,date,sentence,pub,thinktank,sentence_clean,year,international_mention,econ_keyword,edu_keyword,health_keyword,pol_keyword
17646,2025249733.xml,Muslim American candidates hope for 'sweet jus...,2018-04-16,"By last year, just 8 percent of voting America...",WP,Pew Research Center,"By last year, just 8 percent of voting America...",2018,NaN,0,1,0,1


In [208]:
i=thinktank_df[thinktank_df.pol_keyword == 1].sample(1).index
thinktank_df[thinktank_df.pol_keyword == 1]['sentence_clean'][i.item()]

'officials have told some members of Congress that they are certain that Russian hackers were behind the latest breach, two people familiar with these briefings said.              Hackers and spies have long targeted U.S. political parties, but the deliberate leak of party emails and other records this year has created new challenges for law enforcement and intelligence officials as they search for motives and a potential response.              "The incident itself is unprecedented, but they are trying to follow past precedents in their response," such as with Russia\'s alleged recent hacks on White House and State Department systems, where the full U.S. response isn\'t known, said Peter Singer, a senior fellow at the New America think tank and an expert on the use of cyberweapons by nation states.              On Friday, after being dormant for a month, Guccifer 2.0 posted a spreadsheet containing the personal cellphone numbers and personal email addresses of close to 200 current and 